In [12]:
import os
import argparse, sys
import numpy as np
from scipy.integrate import odeint

def pend(y, t,):
    theta, omega = y
    dydt = [omega, - np.sin(theta)]
    return dydt

def pend_damp(y, t,):
    theta, omega = y
    dydt = [omega, - np.sin(theta)-0.1*omega]
    return dydt

def select_ics(theta0,omegaics0):
    ics = []
    for i in range(n_ics):
        for j in range(n_ics):
            lim = (np.abs((omega0[j]**2)/2 - np.cos(theta0[i])))
            if lim <  0.99 :
                ics.append((theta0[i],omega0[j]))
    return ics

def wrap_to_pi(z):
    z_mod = z % (2*np.pi)
    subtract_m = (z_mod > np.pi) * (-2*np.pi)
    return z_mod + subtract_m

def image_gen(ics):
    x = np.linspace(-1.5, 1.5, NX)
    y = np.linspace(-1.5, 1.5, NY)
    xx,yy = np.meshgrid(x, y)


    data = np.empty([len(ics), len(t), len(x), len(y)],dtype = np.float32)
    data2 = np.empty([len(ics), len(t), len(x), len(y)],dtype = np.float32)


    for idx in range(len(ics)):
        if(idx%100==0): print(idx,' su ', len(ics))
        y0 = [ics[idx][0], ics[idx][1]]
        sol = odeint(pend, y0,t)
        theta = sol[:,0]
        omega = sol[:,1]
        print(max(theta))
        temp = []
        for i in range(len(theta)):
            z = np.exp(- 20 *((xx - np.cos(theta[i] + np.pi/2))*(xx - 
                np.cos(theta[i] +np.pi/2))) - 20 * ((yy -np.sin(theta[i]+np.pi/2))*(yy -np.sin(theta[i]+np.pi/2))))
            z = ((z - np.min(z))/(np.max(z)-np.min(z)))

            temp.append(z)
        data[idx] = np.array(temp)
        
        temp = []
        for i in range(len(omega)):
            exp = np.exp(- 20 *((xx - np.cos(omega[i] + np.pi/2))*(xx - 
                np.cos(omega[i] +np.pi/2))) - 20 * ((yy -np.sin(omega[i]+np.pi/2))*(yy -np.sin(omega[i]+np.pi/2))))

            z = -20*(2*(xx - np.cos(theta[i]-np.pi/2))*np.sin(theta[i]-np.pi/2)*omega[i] 
                        + 2*(yy - np.sin(theta[i]-np.pi/2))*(-np.cos(theta[i]-np.pi/2))*omega[i])
            z = z*exp
            z = ((z - np.min(z))/(np.max(z)-np.min(z)))

            temp.append(z)
        data2[idx] = np.array(temp)
        
    return data,data2

In [13]:
n_ics=8
    
#COSTANTI e PARAMETRI
ta = 0.
tb = 5.
dt = 0.05  
# tb = 10
# dt = 0.1
NX = 51
NY = 51


t = np.arange(ta, tb ,dt)
theta0 = np.linspace(-np.pi,np.pi,n_ics)
omega0 = np.linspace(-2.1, 2.1,n_ics)

ics = select_ics(theta0,omega0)
data,data2 = image_gen(ics)


#questo reshape serve per mandare al autoencoder delle immagini flat
#TODO verifica che sia corretto questo rehsape --> dovrebb essere ok fatto prova su colab
data = data.reshape((len(ics) * len(t),NX * NY))
data2 = data2.reshape((len(ics) * len(t),NX * NY))

X = data
Xdot = data2


0  su  28
2.2964315102936497
2.302931523365327
-0.26798332051044205
1.7542347676736891
1.392058781250655
1.3922181731906385
1.754050279302099
2.696168524625678
1.5911858533961094
1.0518428881909678
0.5433259167721869
0.5433335061480165
1.0518460552461335
1.7967385990820735
1.053329776997493
0.9109488513432804
0.4487989505128276
0.5433299046887049
1.0516037993648484
1.7965117600558485
1.3463968515384828
1.3463968515384828
1.3463968515384828
1.3923132517714483
1.7542742734710688
2.696221767202715
2.243994752564138
2.302968188100601


In [11]:
theta0

array([-3.14159265, -2.24399475, -1.34639685, -0.44879895,  0.44879895,
        1.34639685,  2.24399475,  3.14159265])

In [ ]:
def wrap_to_pi(z):
    z_mod = z % (2*np.pi)
    subtract_m = (z_mod > np.pi) * (-2*np.pi)
    return z_mod + subtract_m

In [9]:
wrap_to_pi(theta)

1.0471975511965974